# Jigsaw

- Implement an NxN jigsaw puzzle.
- Design the data structures and explain an algorithm to solve the puzzle.
- You can assume that you have a `fitsWith` method which, when passed two puzzle edges, returns `true` if the two edges belong together.

In [8]:
import numpy as np
import random


pieces = list(range(25))
random.shuffle(pieces)

puzzle_a = np.array(pieces).reshape(5, 5)
puzzle_l = puzzlez.tolist()
puzzle_l

[[21, 10, 15, 11, 2],
 [4, 22, 17, 6, 7],
 [14, 12, 3, 19, 24],
 [16, 8, 5, 23, 1],
 [13, 20, 9, 18, 0]]

In [53]:
from enum import Enum


def fits_with(edge1: "Edge", edge2: "Edge", N: int) -> bool:
    if edge1.is_border() or edge2.is_border():
        return False
    
    locations = {edge1.location, edge2.location}
    vertical_fit = locations == {EdgeLocation.TOP, EdgeLocation.BOTTOM}
    horizontal_fit = locations == {EdgeLocation.LEFT, EdgeLocation.RIGHT}

    if not vertical_fit and not horizontal_fit:
        return False
    
    if vertical_fit:
        if edge1.location == EdgeLocation.TOP:
            return edge1.piece_value == edge2.piece_value - N
        else:  # edge1 is BOTTOM
            return edge2.piece_value == edge1.piece_value + N
    else:
        if edge1.location == EdgeLocation.LEFT:
            return edge1.piece_value == edge2.piece_value - 1
        else:  # edge1 is RIGHT
            return edge2.piece_value == edge1.piece_value + 1


class EdgeLocation(Enum):
    TOP = 0
    LEFT = 1
    BOTTOM = 2
    RIGHT = 3


class Edge:
    def __init__(self, piece_value: int, location: EdgeLocation, N: int):
        self.piece_value = piece_value
        self.location = location
        self.N = N
        self.top_row_values = set(range(N))
        self.bottom_row_values = set(range((N - 1) * N, N * N))
        self.left_col_values = set(range(0, N * N, N))
        self.right_col_values = set(range(N - 1, N * N, N))

    def fits_with(self, other: "Edge") -> bool:
        return fits_with(self, other, self.N)
    
    def is_border(self) -> bool:
        if self.location == EdgeLocation.TOP:
            return self.piece_value in self.top_row_values
        elif self.location == EdgeLocation.BOTTOM:
            return self.piece_value in self.bottom_row_values
        elif self.location == EdgeLocation.RIGHT:
            return self.piece_value in self.right_col_values
        elif self.location == EdgeLocation.LEFT:
            return self.piece_value in self.left_col_values

        
    def __repr__(self):
        return f"{self.piece_value}-{self.location.name}"


class Piece:
    def __init__(self, value: int, N: int):
        self.value = value
        self.top, self.left, self.bottom, self.right = [Edge(value, loc, N) for loc in EdgeLocation]

    def __repr__(self):
        return f"{self.value:02}"


# IDEA: Make them fit by +1 -1 +N -N


class Puzzle:
    def __init__(self, pieces: list[list[int]]):
        # ! Docstring: clarify that pieces should be set by row, as a numpy 2D-array
        self.pieces = np.zeros_like(np.array(pieces)).tolist()
        self.N = len(self.pieces)
        for i, row in enumerate(pieces):
            for j, value in enumerate(row):
                self.pieces[i][j] = Piece(value, self.N)

    def solve(self):
        # TODO Seguir Aca!
        # for row in self.pieces:
        #     for value in row:


    def __repr__(self):
        return "\n".join(str(row) for row in self.pieces)

    def __getitem__(self, ij: tuple[int, int]):
        i, j = ij
        return self.pieces[i][j]

    
puzzle = Puzzle(puzzle_l)
display(puzzle)
puzzle[0, 0].right.fits_with(puzzle[1, 1].left)

[21, 10, 15, 11, 02]
[04, 22, 17, 06, 07]
[14, 12, 03, 19, 24]
[16, 08, 05, 23, 01]
[13, 20, 09, 18, 00]

True

In [54]:
p1 = puzzle[0, 0]
p2 = puzzle[1, 1]

p1, p2

(21, 22)

In [55]:
p1.right, p2.left

(21-RIGHT, 22-LEFT)

In [56]:
p1.right.fits_with(p2.left)

True